# [我爱我家（天津）](https://tj.5i5j.com/) - 租房爬虫
March 19th, 2024.

## 第一步：爬取目录
爬取目录下的所有租房信息，并爬取详情页链接

In [13]:
# 导入所有需要的包
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.edge.options import Options as EdgeOptions
from selenium.common.exceptions import NoSuchElementException
import time
import random
import json
import csv
import os

In [14]:
# 页面的基础URL，使用{}作为页码的占位符
base_url = 'https://tj.5i5j.com/zufang/o8n{}'

# 初始化Edge浏览器的配置选项
edge_options = EdgeOptions()
edge_options.add_argument("--headless")  # 添加headless模式选项
edge_options.add_argument("--disable-gpu")  # 禁用GPU加速，有时在headless模式下需要设置

# 设置Edge浏览器驱动的路径，确保Edge驱动与你的浏览器版本兼容
edge_service = EdgeService(executable_path="./input/msedgedriver")

# 从提供的JSON文件加载User-Agent数据，用于模拟不同的浏览器访问
with open('./input/User-Agents.json') as file:
    user_agents = [agent['ua']
                   for agent in json.load(file)['Desktop Useragents']]

# 填入你的Cookies
cookies_str = """ser_cookie_id=71D36BDF-D93B-6F2F-A98F-27F784FF5BD6; smidV2=2024031900553341bb507e84be121ed62198d61b0bc4c70015991e94de4a330; sensorsdata2015jssdkchannel=%7B%22prop%22%3A%7B%22_sa_channel_landing_url%22%3A%22%22%7D%7D; dd3f54b758ee58c0958869af0974560e_pc=TBYRRFZKXlNdAUFbEANZBQIGVwAGCQUIQ08VXF1RDgtRXVUSChJsRQcCAwFsRQVSUgZsRQcFAghsRQYCAwcBCAAAEhwSRUNVQnlUEgoSAQEIBAgJCQkGBxIcElhVUVRZXVcSChISHBJEX1tVXhIKElVJegBVaHFZf1l6e2YBYVl8c3pYUndTWX9ZenllSnkBfll5Q3ldQABRY3kGeVp1WVZhHlVJekBTA31Zf1l6BlF3dkVqA35YUll5Q3ldQABRY3kGeVp1WXxzekBpaGFZf1p1A31kcQN_dHlIfkpxQ3lddgFqc3kGeV5EU3leZkBqdkdZf1xHWX1kdQR-dFcFf2RbAn4BR1l8dkdZUncJXlFnBUVpZwFcaHN5BmhzelN5WUhTeV0FQGkCREVpZwFcaHN5BmhzelNoeGUDfVp9SGh2SAF-Z3pZflxIU1RkUwF9WlhTaHhlAn1afQN9ZFdHfXZHWXx2R1lqZwFYUWdIU3laQEVUZ0hDfHZHWWoCZkVqd2ZJaHN5BlJeZkNSc0hTeV56XGoCXEZSXEdZf10FAVJ3R0Noc3paUWhiBWhzeQZoc3kDaHN5Q2hzel9qZ3ZbUWcBXmhzeQZSXmZDUnNIU3ldXEVUXVwAUWcFXmkCCVtqZkdZf10FAVJ3R0Noc3pJamdUW2loYlxoc3kGaHN5SH5KdUd-SldJfWRTR2hzeUNoc3pKVHd2AFRoflN5Wl9IfHZHWVN3dkpTAVRGU11iU3laQFN5XX5dfWdlAn5nfQJ9XXkAfnR1SH53el19XWZZaWRpSH0CYQR-WlsFand5BGhzeUNoc3pKaWdIAGhzeQZoc3lIfVp9AH5kalN5WUhTeV5mSmpoekRSAnpAUndmU3laQFN5WnUEfVp1A31afQJ_ZHFHaHN5Q2hzegBRZwFcUwNiWFJoclN5Wl9Ifkp1R35KV0l9ZFNHVmN5Q3lefgFpWXkGeV5-AWldQFxpA2FZVmEedn9bWn5zXW9Relx1dURfRW9_X29UfWVRRWFTan15SUJXZ1sBfgl6Sn0AeRIcEkNRXEQSChIBAgMEBQYSTQ; wiwj_token_202_113_13_148_17107812001184899967=%7B%22uid%22%3A%221184899967%22%7D; tdCookieUid=1184899967; sensorsdata2015jssdkcross=%7B%22distinct_id%22%3A%221184899967%22%2C%22first_id%22%3A%2218e527d67d91349-02afdd2a11d6d7e-7e433c49-1405320-18e527d67da9d0%22%2C%22props%22%3A%7B%22%24latest_traffic_source_type%22%3A%22%E7%9B%B4%E6%8E%A5%E6%B5%81%E9%87%8F%22%2C%22%24latest_search_keyword%22%3A%22%E6%9C%AA%E5%8F%96%E5%88%B0%E5%80%BC_%E7%9B%B4%E6%8E%A5%E6%89%93%E5%BC%80%22%2C%22%24latest_referrer%22%3A%22%22%2C%22_latest_wx_ad_click_id%22%3A%22%22%2C%22_latest_wx_ad_hash_key%22%3A%22%22%2C%22_latest_wx_ad_callbacks%22%3A%22%22%7D%2C%22identities%22%3A%22eyIkaWRlbnRpdHlfY29va2llX2lkIjoiMThlNTI3ZDY3ZDkxMzQ5LTAyYWZkZDJhMTFkNmQ3ZS03ZTQzM2M0OS0xNDA1MzIwLTE4ZTUyN2Q2N2RhOWQwIiwiJGlkZW50aXR5X2xvZ2luX2lkIjoiMTE4NDg5OTk2NyJ9%22%2C%22history_login_id%22%3A%7B%22name%22%3A%22%24identity_login_id%22%2C%22value%22%3A%221184899967%22%7D%2C%22%24device_id%22%3A%2218e527d67d91349-02afdd2a11d6d7e-7e433c49-1405320-18e527d67da9d0%22%7D; historyCity=%5B%22%5Cu4e0a%5Cu6d77%22%2C%22%5Cu5317%5Cu4eac%22%2C%22%5Cu5929%5Cu6d25%22%5D; zufang_BROWSES=90012074%2C90011689%2C90005422%2C90005456%2C819266853%2C819344107%2C819265997%2C90014705%2C39605346%2C824590282; wafatcltime=2851944; wafatclconfirm=D+Q0TY3XkWd/d4kyLKHIZL75JnV0Tw86gbYwrBEE1oX2Gs10Y3V94K+L4hB2ehaNfOkazsWI7y7Ekd6Tu+9A9uMj/YjlsObNSW49PEqvLgWBtoSUmTntGbJMn1AHzE+3yBNbYgfo+wQw3cW16bfX1mUqmGJjwGG2lw+7i6102XQ8PPX8RYU7npdllDJjLm9o/xLX1gMOwcL/tRFgCMRO7z5sbRNeZsQhzKRvX91qa963Rvful8vLv4V6QgXTZUlOmw4BDCTzWu245MayJeU55k5nyeeUTCjJUSeyaGT4uur3zUvD34lr6KC9NY3UQEQZoANfK+Ozv5LVXqoBzj08GUUf6ZqQdi5kw9hhverove47x+az0qkgUU3add0OSYnDJKserzFZXdNjNCQPwoFlCRPTZctBoV5ISFRdt7UE04hrkK4Ffz4E1hN/G+YWa+6KLsFMoARKN7w3BGnpZ2WkwRAlMWUWj9Z1wro42RY8lwtsSwIsmxNl0KRjg3wpZRh+Fn6Udd7MmwSikzgqdY8MzDL87muI8MerKKTSEfsP9UGYFphKO/o2q0kvHk+kNjYOv28eIQdOqtnlbmX+Oynl8aFgn917gbNweXQCloNZKcWHG88ombIkuP1+ut9uKvge39fhOhtArdHnGnfMVv7MepbirR+t0mK3n8MmXsFc5vakxlWS8hKduYCOSYJTFVXQabcum/X4fHkCDNWGTw6zRKDisaRi9L0MKExSvoinWOY%3D; wafatcltoken=4a829a89e53164cd16a49ffb85d95c6a; HWWAFSESID=a55278b0cd730207b6; HWWAFSESTIME=1711166865026; PHPSESSID=jmmp2ff2n6tpba3tu0m3qds58l; domain=sh; 39d1b036ea78c3b6a3815611756ab441_pc=TBYRRFZKXlNdAUFbEANZBQIGVwAGCQUIQ08VXF1RDgtRXVUSChJsRQcCAwFsRQVSUgZsRQcFAghsRQYCAwcBCAAAEhwSRUNVQnlUEgoSAQEIBAgJCQkGBxIcElhVUVRZXVcSChISHBJEX1tVXhIKElVJegBVaHFZf1l6e2YBYVl8c3pYUndTWX9ZenllSnkBfll5Q3ldQABRY3kGeVp1WVZhHlVJekBTA31Zf1l6BlF3dkVqA35YUll5Q3ldQABRY3kGeVp1WXxzekBpaGFZf1p1A31kdUh-WlNHf2R9Q3lddgFqc3kGeV5EU3leZkBqdkdZf1xHWX1kdQR-dFcFf2RbAn4BR1l8dkdZUncJXlFnBUVpZwFcaHN5BmhzelN5WUhTeV0FQGkCREVpZwFcaHN5BmhzelNoeGUDfVp9SGh2SAF-Z3pZflxIU1RkUwF9WlhTaHhlAn1afQN9ZFdHfXZHWXx2R1lqZwFYUWdIU3laQEVUZ0hDfHZHWWoCZkVqd2ZJaHN5BlJeZkNSc0hTeV56XGoCXEZSXEdZf10FAVJ3R0Noc3paUWhiBWhzeQZoc3kDaHN5Q2hzel9qZ3ZbUWcBXmhzeQZSXmZDUnNIU3ldXEVUXVwAUWcFXmkCCVtqZkdZf10FAVJ3R0Noc3pJamdUW2loYlxoc3kGaHN5SH5KdUd-SldJfWRTR2hzeUNoc3pKVHd2AFRoflN5Wl9IfHZHWVN3dkpTAVRGU11iU3laQFN5XX5dfWdlAn5nfQJ9XXkAfnR1SH53el19XWZZaWRpSH0CYQR-WlsFand5BGhzeUNoc3pKaWdIAGhzeQZoc3lIfVp9AH5kalN5WUhTeV5mSmpoekRSAnpAUndmU3laQFN5WnUEfVp1A31afQJ_ZHFHaHN5Q2hzegBRZwFcUwNiWFJoclN5Wl9Ifkp1SH1kaQN9dFtKVmN5Q3lefgFpWXkGeV5-AWldQFxpA2FZVmEeU2ZBBmloA3J-eFZKeXRUA0B2emYBSWdqYgBpBlRJekpdRkkABmVRAFFxaRIcEkNRXEQSChIBAgMEBQYSTQ; user_info=TBYRRFZKXlNdAUFbEANZBQIGVwAGCQUIQ08VXF1RDgtRXVUSChJsRQcCAwFsRQVSUgZsRQcFAghsRQYCAwcBCAAAEhwSRUNVQnlUEgoSAQEIBAgJCQkGBxIcElhVUVRZXVcSChISHBJEX1tVXhIKElVJegBVaHFZf1l6e2YBYVl8c3pYUndTWX9ZenllSnkBfll5Q3ldQABRY3kGeVp1WVZhHlVJekBTA31Zf1l6BlF3dkVqA35YUll5Q3ldQABRY3kGeVp1WXxzekBpaGFZf1p1A31kdUh-WlNHf2R9Q3lddgFqc3kGeV5EU3leZkBqdkdZf1xHWX1kdQR-dFcFf2RbAn4BR1l8dkdZUncJXlFnBUVpZwFcaHN5BmhzelN5WUhTeV0FQGkCREVpZwFcaHN5BmhzelNoeGUDfVp9SGh2SAF-Z3pZflxIU1RkUwF9WlhTaHhlAn1afQN9ZFdHfXZHWXx2R1lqZwFYUWdIU3laQEVUZ0hDfHZHWWoCZkVqd2ZJaHN5BlJeZkNSc0hTeV56XGoCXEZSXEdZf10FAVJ3R0Noc3paUWhiBWhzeQZoc3kDaHN5Q2hzel9qZ3ZbUWcBXmhzeQZSXmZDUnNIU3ldXEVUXVwAUWcFXmkCCVtqZkdZf10FAVJ3R0Noc3pJamdUW2loYlxoc3kGaHN5SH5KdUd-SldJfWRTR2hzeUNoc3pKVHd2AFRoflN5Wl9IfHZHWVN3dkpTAVRGU11iU3laQFN5XX5dfWdlAn5nfQJ9XXkAfnR1SH53el19XWZZaWRpSH0CYQR-WlsFand5BGhzeUNoc3pKaWdIAGhzeQZoc3lIfVp9AH5kalN5WUhTeV5mSmpoekRSAnpAUndmU3laQFN5WnUEfVp1A31afQJ_ZHFHaHN5Q2hzegBRZwFcUwNiWFJoclN5Wl9Ifkp1SH1kaQN9dFtKVmN5Q3lefgFpWXkGeV5-AWldQFxpA2FZVmEeU2ZBBmloA3J-eFZKeXRUA0B2emYBSWdqYgBpBlRJekpdRkkABmVRAFFxaRIcEkNRXEQSChIBAgMEBQYSTQ; wiwj_token_ticket=202_113_13_3_17111664001184899967; wiwj_token_202_113_13_3_17111664001184899967=%7B%22uid%22%3A%221184899967%22%7D; uid=1184899967; userToken=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiIsImp0aSI6IjEifQ.eyJpc3MiOiJ6aGFuZ3NhbiIsImp0aSI6IjEiLCJpYXQiOjE3MTExNjcwOTMsImF1ZCI6IntcInVpZFwiOlwiMTE4NDg5OTk2N1wiLFwibG9naW5uYW1lXCI6XCJcIixcIm5pY2tuYW1lXCI6XCJcXHU3MjMxXFx1NWJiNlxcdTc1MjhcXHU2MjM3MTgwMFwiLFwiZW1haWxcIjpudWxsLFwiZ2VuZGVyXCI6bnVsbCxcInJlZ2lvblwiOm51bGwsXCJjaXR5XCI6XCI3XCIsXCJoZWFkaW1nXCI6bnVsbCxcImludml0aW5nY29kZVwiOm51bGwsXCJyZWdkYXRlXCI6XCIxNzEwNzgyMTcwXCIsXCJzdGF0dXNcIjoxLFwicGFzc1dvcmRcIjpcImNmMWU2NWM2MmI0NDExNGJmMmViYTYxM2Q4Njk5ZGI4XCIsXCJzYWx0XCI6XCIxMjM0NTZcIixcInVzZXJtb2JpbGVcIjpcIjE4MjE3MjM2OTAwXCIsXCJ0aW1lc3RhbXBcIjoxNzExMTY3MDkzfSIsInN1YiI6InN1YmplY3QifQ.cVq6YX3BNHfzIDd3pFJV1yWZR0Y6dyJzmvy06Ua0aAY"""

# 将字符串形式的cookies转换为字典形式，方便后续使用
cookies = {cookie.split('=')[0]: cookie.split('=')[1]
           for cookie in cookies_str.split('; ')}

In [15]:
def scrape_rental_list(page_number):
    """
    爬取指定页码的租房列表信息。
    
    该函数通过Selenium WebDriver访问并爬取指定页面的租房信息，包括标题、链接、价格、详情和关注度。
    使用随机的用户代理（User-Agent）访问页面，并在访问前设置cookies以模拟正常用户的访问。

    Args:
        page_number (int): 要爬取的页码。

    Returns:
        list of dict: 爬取到的租房信息列表，每个元素是一个包含标题、链接、价格等信息的字典。
    """
    print(f"fetching from page {page_number}")
    # 随机选择一个用户代理（User-Agent）添加到浏览器选项中
    edge_options.add_argument("user-agent=" + random.choice(user_agents))

    # 初始化WebDriver，即Edge浏览器
    driver = webdriver.Edge(options=edge_options, service=edge_service)

    # 访问指定网站，以便能够设置cookies
    driver.get("https://tj.5i5j.com")
    # 遍历并添加cookies到浏览器会话中
    for name, value in cookies.items():
        driver.add_cookie({"name": name, "value": value})

    # 构造要抓取的页面的URL
    url = base_url.format(page_number)
    # 访问该URL
    driver.get(url)

    # 等待页面加载完成，确保所需元素已经加载出来
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(
            (By.XPATH, '/html/body/div[6]/div[1]/div[2]/ul/li'))
    )

    # 查找页面上所有的列表项元素
    items = driver.find_elements(
        By.XPATH, '/html/body/div[6]/div[1]/div[2]/ul/li')
    # 初始化用于存储每个列表项信息的列表
    page_data = []
    # 遍历每个列表项，抓取所需信息
    for item in items:
        # 获取标题元素及其文本（标题）和链接（href属性）
        title_element = item.find_element(
            By.XPATH, './/div[@class="listCon"]/h3/a')
        title = title_element.text
        title_url = title_element.get_attribute('href')

        # 如果标题和标题链接都不为空，则收集相关信息
        if title and title_url:
            price = item.find_element(
                By.XPATH, './/div[@class="jia"]/p[@class="redC"]/strong').text + "元/月"
            details = item.find_element(
                By.XPATH, './/div[@class="listCon"]/div[@class="listX"]/p[1]').text
            attention = item.find_element(
                By.XPATH, './/div[@class="listCon"]/div[@class="listX"]/p[3]').text

            # 将收集到的信息作为一个字典添加到列表中
            page_data.append({
                'title': title,
                'title_url': title_url,
                'price': price,
                'details': details.strip(),
                'attention': attention.strip(),
                'source_url': url  # 记录当前信息来源的页面URL
            })
    driver.quit()
    return page_data

In [16]:
def save_to_csv_list(page_data, csv_file_path, field_names):
    """
    将爬取的页面数据保存到指定的CSV文件中。

    Args:
        page_data (list of dict): 包含页面数据的列表，每个元素是一个字典。
        csv_file_path (str): 要保存数据的CSV文件的路径。
        field_names (list of str): CSV列的名称。
    """
    # 检查CSV文件是否存在以及是否为空，以决定是否写入表头
    write_header = not os.path.exists(
        csv_file_path) or os.path.getsize(csv_file_path) == 0
    with open(csv_file_path, mode='a', newline='', encoding='utf-8') as file:
        # 定义CSV文件中的列名
        writer = csv.DictWriter(file, fieldnames=field_names)
        # 如果需要，写入表头
        if write_header:
            writer.writeheader()
        # 如果page_data是None，使用空列表来避免TypeError
        if page_data is None:
            page_data = []
        # 遍历页面数据，并将每条数据写入CSV文件
        for data in page_data:
            writer.writerow(data)

In [17]:
# saving_path = "./output/Renting_new.csv"
# field_names = ["title", "title_url", "price",
#                "details", "attention", "source_url"]

# starting_page = 46
# # 请尝试
# ending_page = 900

# for page in range(starting_page, ending_page):
#     renting_data = scrape_rental_list(page)
#     save_to_csv_list(renting_data, saving_path, field_names)
#     time.sleep(15)

## 第二步：数据清理
对第一步爬取的数据（Renting.csv）进行清洗

In [18]:
import pandas as pd

file_path = './input/我爱我家-租房数据.csv'

# Specify that all columns should be read as string data
renting_data = pd.read_csv(file_path, encoding='utf-8', dtype=str)

house_id_list = renting_data['房源编号'].tolist()
house_id_list.reverse()

In [19]:
def scrape_rental_details(listing_number):
    """从指定的房源编号中爬取并解析出租房详情。

    这个函数会访问我爱我家房源，根据提供的房源编号（listing_number），
    爬取该房源的详细信息，如房源亮点、户型介绍、交通出行等。然后，它会使用selenium webdriver来模拟浏览器行为，
    如页面导航和元素选择，以提取需要的信息。

    Args:
        listing_number (str): 房源编号，用于构造房源详细信息的URL。

    Returns:
        dictionary: 爬出的数据
    """
    # 构造房源详细信息页面的URL
    url_detail = f"https://tj.5i5j.com/zufang/{listing_number}.html"
    # print(f"fetching from House {listing_number}, Link => {url_detail}")

    # 设置浏览器驱动和cookie等，用于模拟用户的浏览器行为
    edge_options.add_argument("user-agent=" + random.choice(user_agents))
    driver = webdriver.Edge(options=edge_options, service=edge_service)
    driver.get("https://tj.5i5j.com")
    for name, value in cookies.items():
        driver.add_cookie({"name": name, "value": value})
    driver.get(url_detail)
    time.sleep(10)

    # 初始化存储爬取信息的字典
    renting_details = {
        "房源编号": listing_number,
        "房源亮点": "None",
        "户型介绍": "None",
        "交通出行": "None",
        "周边配套": "None",
        "小区信息": "None",
        "洗衣机": "None",
        "冰箱": "None",
        "电视": "None",
        "空调": "None",
        "热水器": "None",
        "天然气": "None",
        "暖气": "None",
        "床": "None",
        "网络": "None",
        "衣柜": "None",
        "交通": "None",
        "生活": "None",
        "品质": "None",
        "医疗": "None",
        "运动": "None",
        "最低参考价": "None",
        "最高参考价": "None"
    }

    # 使用Selenium WebDriver的find_element方法根据class名定位网页中的元素，并提取小区的最高（低）参考价信息
    try:
        # 通过CSS类名"minprice"定位到表示最高（低）参考价的元素
        min_price_element = driver.find_element(By.CLASS_NAME, "minprice")
        min_price = min_price_element.find_element(By.TAG_NAME, "span").text

        # 在该元素内部，进一步通过标签名"span"定位到包含具体价格信息的子元素，并获取其文本内容作为最高（低）参考价
        max_price_element = driver.find_element(By.CLASS_NAME, "maxprice")
        max_price = max_price_element.find_element(By.TAG_NAME, "span").text
        renting_details["最低参考价"] = int({min_price}.pop())
        renting_details["最高参考价"] = int({max_price}.pop())
    except:
        pass

    # 使用Selenium WebDriver的find_element方法根据class名定位网页中的元素，并提取小区的房源特色栏目信息
    try:
        # 通过CSS类名"fytese"定位到表示房源特色栏目的元素
        detail_elements = driver.find_elements(By.CLASS_NAME, "fytese")
        for detail in detail_elements:
            # 在每个特色信息元素内，通过标签名"li"找到所有列表项。
            items = detail.find_elements(By.TAG_NAME, "li")
            for item in items:
                # 在列表项内，通过标签名"span"找到标签元素，并获取房源特色名称。
                label = item.find_element(By.TAG_NAME, "span").text.strip()
                # 在列表项内，通过标签名"label"找到值元素，并获取房源特色的具体信息。
                value = item.find_element(By.TAG_NAME, "label").text.strip()
                if label in renting_details:
                    renting_details[label] = value
    except:
        pass

    # 使用Selenium WebDriver的find_element方法根据class名定位网页中的元素，并提取小区的房源配套设施信息
    try:
        # 通过CSS类名"fysty"定位到表示房源特色栏目的元素
        facilities_list = driver.find_element(By.CLASS_NAME, "fysty")
        facilities_items = facilities_list.find_elements(By.TAG_NAME, "li")
        for item in facilities_items:
            # 在每个配套设施项内部，进一步定位到包含特定属性（如是否可用）的span元素
            # 并获取该span元素的class属性值
            facility_name = item.text
            # 根据span元素的class属性值判断该设施是否可用
            # 如果class属性值中不包含"-1"，则认为该设施可用，否则不可用
            # 将结果以设施名称为键，可用性（布尔值）为值存入renting_details字典中
            span_class = item.find_element(
                By.TAG_NAME, "span").get_attribute("class")
            renting_details[facility_name] = "-1" not in span_class
    except:
        pass

    # 使用Selenium爬取不同分类（如交通、生活、品质、医疗、运动）的信息。
    # 定义要抓取的信息分类列表
    categories = ['交通', '生活', '品质', '医疗', '运动']

    # 对每一个分类执行数据抓取
    for category in categories:
        try:
            # 定位分类选项卡和内容的Xpath路径
            zhishu_tab_xpath = "//div[@class='zhishu_tab']"
            zhishu_content_xpath = "//div[@class='zhishu_content']"

            # 使用Xpath和类别名称找到对应的分类标签并点击，切换至相应内容
            tab = driver.find_element(
                By.XPATH, f"//ul[@class='zhishu_menus']/li[contains(text(), '{category}')]")
            tab.click()
            WebDriverWait(driver, 1).until(
                EC.presence_of_element_located((By.XPATH, zhishu_content_xpath)))

           # 找到并收集当前分类下所有信息内容的文本
            content_divs = driver.find_elements(
                By.XPATH, f"{zhishu_content_xpath}/div[contains(@class, 'zhishu_main')]")
            category_content = []
            for div in content_divs:
                category_content.append(div.text)

            non_empty_content = next(
                (item for item in category_content if item), None)

            # 筛选出第一个非空的内容项
            if non_empty_content:
                cleaned_content = non_empty_content.strip()
                cleaned_content = cleaned_content.replace('\n', ' ')
                # 清除不必要的字符串
                if cleaned_content.endswith("查看更多周边信息"):
                    cleaned_content = cleaned_content[:-
                                                      len("查看更多周边信息")].rstrip()
                renting_details[category] = cleaned_content
        except Exception as e:
            pass
    driver.quit()
    return renting_details

In [20]:
def save_to_csv_detail(rental_detail, csv_file_path, field_names):
    """
    将单个租房详情字典保存到CSV文件中。

    Args:
        rental_detail (dict): 单个租房详情字典。
        csv_file_path (str): CSV文件的保存路径。
        field_names (list of str): CSV文件中列的名称列表。
    """
    # 确保提供的租房详情不是None，如果是None，则不进行任何操作
    if rental_detail is None:
        return

    # 检查CSV文件是否存在以及是否为空，以决定是否写入表头
    write_header = not os.path.exists(csv_file_path) or os.path.getsize(csv_file_path) == 0

    with open(csv_file_path, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=field_names)
        if write_header:
            writer.writeheader()
        writer.writerow(rental_detail)

In [21]:
# scrape_rental_details(random.choice(house_id_list))
# scrape_rental_details(819339985) # 全都有
# scrape_rental_details(819303631)  # 全都有
rental_details_path = "./output/rental_details.csv"
details_field_names = ['房源编号', '房源亮点', '户型介绍', '交通出行',
                       '周边配套', '小区信息', '洗衣机', '冰箱', '电视',
                       '空调', '热水器', '天然气', '暖气', '床',
                       '网络', '衣柜', '交通', '生活',
                       '品质', '医疗', '运动', '最低参考价',
                       '最高参考价']

start_index = 0

for i in range(start_index, len(house_id_list)):
    id = house_id_list[i]
    print(f"Processing index {i}, ID: {id}")  # 打印当前索引和ID
    rental_details = scrape_rental_details(id)
    save_to_csv_detail(rental_details, rental_details_path,
                       details_field_names)
    print(rental_details)
    # time.sleep(10)

Processing index 0, ID: 90014705
{'房源编号': '90014705', '房源亮点': 'None', '户型介绍': 'None', '交通出行': 'None', '周边配套': 'None', '小区信息': 'None', '洗衣机': 'None', '冰箱': 'None', '电视': 'None', '空调': 'None', '热水器': 'None', '天然气': 'None', '暖气': 'None', '床': 'None', '网络': 'None', '衣柜': 'None', '交通': 'None', '生活': 'None', '品质': 'None', '医疗': 'None', '运动': 'None', '最低参考价': 'None', '最高参考价': 'None'}
Processing index 1, ID: 90014704
{'房源编号': '90014704', '房源亮点': '相寓深植房屋资产管理领域，持续打造广布全国的高品质租住空间，海量房源，多元产品选择，以贴心品质服务为客户缔造美好租+生活', '户型介绍': 'None', '交通出行': '距离地铁建昌道站直线距离364米', '周边配套': 'None', '小区信息': 'None', '洗衣机': True, '冰箱': True, '电视': True, '空调': True, '热水器': False, '天然气': False, '暖气': False, '床': True, '网络': False, '衣柜': True, '交通': '距 5号线 建昌道地铁站-D口 486米 距 春和仁居 227米 路网畅通', '生活': '500米内 平价餐厅64家 ，7家4星餐厅 500米内 3家便利店 1000米内 45家综合超市 2000米内 4家商场 500米内 药店 6家 1000米内 银行 6家', '品质': '小区物业费1.8元/平米/月 物业公司为天津市华海物业管理有限公司 绿化率40% 容积率1.6', '医疗': '2000米内二级及以下医院 1家 距离 天津市河北区建昌医院 1053米', '运动': '距离 附近最近公园 2000米 1000米内 1家 健身房 距离最近健身房金吉鸟

KeyboardInterrupt: 